In [1]:
from text_utils import *

#current version
#export TF_BINARY_URL=https://storage.googleapis.com/tensorflow/linux/gpu/tensorflow_gpu-1.0.1-cp36-cp36m-linux_x86_64.whl
import tensorflow as tf
from tensorflow.contrib.rnn.python.ops import core_rnn_cell as rnn_cell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn,dynamic_rnn
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pickle as pkl
import pandas as pd
from os import walk
import h5py
import sys
sys.path.append('../tfmodels')
#from tf_object import *
from text_cnn import *
from collections import Counter


In [2]:
"""
sentence_ids = []
label = []
id_term_map = {}
for idx, file in enumerate(['neg','pos']):
    sentences = []
    file_name = 'sentiment/{}.txt'.format(file)
    with open(file_name,'r') as f:
        print(file_name, len(id_term_map))
        for line in f:
            sentence_ids.append(update_bow(weibo_filter(line.strip()), id_term_map, terminator=0))
            label.append(idx)
"""

"\nsentence_ids = []\nlabel = []\nid_term_map = {}\nfor idx, file in enumerate(['neg','pos']):\n    sentences = []\n    file_name = 'sentiment/{}.txt'.format(file)\n    with open(file_name,'r') as f:\n        print(file_name, len(id_term_map))\n        for line in f:\n            sentence_ids.append(update_bow(weibo_filter(line.strip()), id_term_map, terminator=0))\n            label.append(idx)\n"

In [3]:
"""
sentence = []
label = []
for (dirpath, dirnames, filenames) in walk('my_data/labeled_data/'):
    for file in filenames:
        word_df = pd.read_excel('my_data/labeled_data/{}'.format(file))
        sentence += word_df['content'].tolist()
        label += word_df['label'].tolist()

sentence_ids = []
id_term_map = {}

label_dict = {'中性':0,'正面':1,'负面':2, '无关':3}  
label = [label_dict[i] for i in label]
for line in sentence:
    sentence_ids.append(update_bow(weibo_filter(line.strip()), id_term_map, terminator=0))
"""

"\nsentence = []\nlabel = []\nfor (dirpath, dirnames, filenames) in walk('my_data/labeled_data/'):\n    for file in filenames:\n        word_df = pd.read_excel('my_data/labeled_data/{}'.format(file))\n        sentence += word_df['content'].tolist()\n        label += word_df['label'].tolist()\n\nsentence_ids = []\nid_term_map = {}\n\nlabel_dict = {'中性':0,'正面':1,'负面':2, '无关':3}  \nlabel = [label_dict[i] for i in label]\nfor line in sentence:\n    sentence_ids.append(update_bow(weibo_filter(line.strip()), id_term_map, terminator=0))\n"

In [2]:
f = open('emotion_sents.dat','rb')
new_sentences = pkl.load(f)
label_str = pkl.load(f)
f.close()
df = pd.read_excel('emotion.xlsx')
maxlen = 140
phrase_dict = dict(zip(df.phrase, df.po1))
id_term_map = pkl.load(open('term_map.dat','rb'))
term_dict = dict((v,k) for k,v in id_term_map.items())

In [3]:
#label_dict = {x:i for i,x in enumerate(set(label))}
label_dict = dict(zip(df.phrase,df.po2))
label = [label_dict[i] for i in label_str]

sentence_ids = []
id_term_map = {}
for line in new_sentences:
    sentence_ids.append(update_bow(weibo_filter(line.strip()), id_term_map, terminator=0))
pkl.dump(id_term_map, open('term_map.dat','wb'))

#seq_len = np.array([len(s)-1 for s in sentence_ids]) 
#new_idx = np.arange(0,len(seq_len))[np.logical_and(seq_len<maxlen, seq_len>0)]
seq_len = np.array([len(s) for s in sentence_ids]) 
new_idx = np.arange(0,len(seq_len))[seq_len<maxlen]
seq_len = seq_len[new_idx]
label = np.array(label)[new_idx]
documents = np.empty(shape=(len(seq_len),maxlen))
for i, idx in enumerate(new_idx):
    if i%100000==0:
        print(i)
    documents[i] = np.pad(np.array(sentence_ids[idx]),((0),(maxlen-seq_len[i])), mode='constant', constant_values=0)  
label += 2

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000


In [4]:
file_path = 'emotion_documents'
h5f = h5py.File(file_path + '.h5', 'w')
h5f.create_dataset('documents', data=documents)
h5f.create_dataset('seq_len', data=seq_len)
h5f.create_dataset('label', data=label)
h5f.create_dataset('new_idx', data=new_idx)
h5f.create_dataset('id_term_map_length', data=len(id_term_map))
h5f.close()


In [5]:
file_path = 'emotion_documents'
h5f = h5py.File(file_path + '.h5', 'r')
documents = np.array(h5f['documents'])
seq_len = np.array(h5f['seq_len'])
label = np.array(h5f['label'])
new_idx = np.array(h5f['new_idx'])
nb_words = np.array(h5f['id_term_map_length'])
h5f.close()
label_str = [label_str[idx] for idx in new_idx]

In [6]:
Counter(label)

Counter({0: 474546, 1: 2084242, 2: 1380949, 3: 2024276, 4: 1846890})

In [14]:
flags = tf.app.flags

flags.DEFINE_integer("nb_words", nb_words, "term number in input sequence(zero mask) [20001]")
flags.DEFINE_integer("maxlen", maxlen, "the max length of input sequence [80]")
flags.DEFINE_integer("embedding_size", 300, "word embedding size [50]")
flags.DEFINE_float("init_scale", 1.0, "init scale for embedding layer")
flags.DEFINE_float("learning_rate", 0.001, "learning rate [0.001]")
flags.DEFINE_integer("batch_size", 512, "batch size to use during training [128]")
flags.DEFINE_float("clip_gradients", 5.0, "clip gradients to this norm [5.0]")
flags.DEFINE_integer("n_epochs", 50, "number of epoch to use during training [10]")
flags.DEFINE_boolean("epoch_save", True, "save checkpoint or not in each epoch [True]")
flags.DEFINE_integer("print_step", 100, "print step duraing training [100]")
flags.DEFINE_string("logs_dir", "logs/", "logs directory [logs/]")
flags.DEFINE_string("model_dir", "model/", "model directory [model/]")
flags.DEFINE_boolean("dir_clear", False, "clear the log and model directory")
flags.DEFINE_boolean("lr_annealing", False, "use lr annealing or not after each epoch [False]")
flags.DEFINE_string("current_task_name", 'url_self_prediction', "current task name [self_prediction]")
flags.DEFINE_integer("gpu_id", 0, "default gpu id [0]")
flags.DEFINE_integer("gpu_num", 4, "gpu_num")

FLAGS = flags.FLAGS

In [15]:
idxs = np.arange(0, len(seq_len))
label[label<=1] = 0
label[label>=3] = 1
#2330054
idxs = np.append(np.random.choice(np.where(label == 0)[0], 437659, replace=False),
                 np.random.choice(np.where(label == 1)[0], 437659, replace=False))
#idxs = np.delete(idxs, np.where(label == 2)[0])
#idxs = np.delete(idxs, np.where(label == 3)[0])
#idxs = idxs[0:200000]
train_idxs, test_idxs = train_test_split(idxs, test_size=0.2, random_state=42)
test_idxs = np.sort(test_idxs)

In [16]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    #cnn_model = TextCNN(FLAGS, session, current_task_name='text_cnn_model')
    #cnn_model.build_model(num_classes=len(set(label[idxs])),max_conv_len=7, num_filters=512, dropout_keep_prob=0.5)
    cnn_model = TextMultiCNN(FLAGS, session, current_task_name='text_multi_cnn_model')
    #label[idxs]
    cnn_model.build_model(num_classes = len(set(label[idxs])), num_filters_per_size=256, fully_layers = [1024, 1024])
    cnn_model.build_model_summary()
    display(cnn_model.model_summary())
    cnn_model.run([documents,label], train_idxs, test_idxs)

Initializing


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1.0
1,gpu/embedding/embedding_layer/embedding_table:0,"[15987, 300]",4796100.0
2,gpu/conv/conv_1d/he_uniform/W:0,"[7, 300, 256]",537600.0
3,gpu/conv/conv_1d/B:0,[256],256.0
4,gpu/conv_1/conv_1d/he_uniform/W:0,"[7, 256, 256]",458752.0
5,gpu/conv_1/conv_1d/B:0,[256],256.0
6,gpu/conv_2/conv_1d/he_uniform/W:0,"[3, 256, 256]",196608.0
7,gpu/conv_2/conv_1d/B:0,[256],256.0
8,gpu/conv_3/conv_1d/he_uniform/W:0,"[3, 256, 256]",196608.0
9,gpu/conv_3/conv_1d/B:0,[256],256.0


Epoch 1 ... training ...
Minibatch 100 / loss: 0.698411
Minibatch 100 / accuracy: 0.474609
Minibatch 200 / loss: 0.695136
Minibatch 200 / accuracy: 0.494141
Minibatch 300 / loss: 0.69388
Minibatch 300 / accuracy: 0.478516
Minibatch 400 / loss: 0.693801
Minibatch 400 / accuracy: 0.505859
Minibatch 500 / loss: 0.694499
Minibatch 500 / accuracy: 0.490234
Minibatch 600 / loss: 0.691916
Minibatch 600 / accuracy: 0.53125
Minibatch 700 / loss: 0.695897
Minibatch 700 / accuracy: 0.492188
Minibatch 800 / loss: 0.693354
Minibatch 800 / accuracy: 0.498047
Minibatch 900 / loss: 0.69398
Minibatch 900 / accuracy: 0.488281
Minibatch 1000 / loss: 0.697048
Minibatch 1000 / accuracy: 0.462891
Minibatch 1100 / loss: 0.697587
Minibatch 1100 / accuracy: 0.478516


KeyboardInterrupt: 

In [7]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    cnn_model = TextMultiCNN(FLAGS, session, current_task_name='text_multi_cnn_model')
    #label[idxs]
    cnn_model.build_model(num_classes = len(set(label[idxs])), num_filters_per_size=256, fully_layers = [1024, 1024])
    cnn_model.build_model_summary()
    cnn_model.model_restore()
    display(cnn_model.model_summary())
    _, testMetric, results = cnn_model.model_run([documents,label], 
                                                  test_idxs, run_type='self',
                                                 mode='test', shuffle=False, save_metric=True)

Initializing
INFO:tensorflow:Restoring parameters from model/text_multi_cnn_model.ckpt
Load Model ...


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1.0
1,gpu/embedding/embedding_layer/embedding_table:0,"[15987, 300]",4796100.0
2,gpu/conv/conv_1d/he_uniform/W:0,"[7, 300, 256]",537600.0
3,gpu/conv/conv_1d/B:0,[256],256.0
4,gpu/conv_1/conv_1d/he_uniform/W:0,"[7, 256, 256]",458752.0
5,gpu/conv_1/conv_1d/B:0,[256],256.0
6,gpu/conv_2/conv_1d/he_uniform/W:0,"[3, 256, 256]",196608.0
7,gpu/conv_2/conv_1d/B:0,[256],256.0
8,gpu/conv_3/conv_1d/he_uniform/W:0,"[3, 256, 256]",196608.0
9,gpu/conv_3/conv_1d/B:0,[256],256.0


Minibatch 1000 / loss: 0.197946
Minibatch 1000 / accuracy: 0.943359


In [8]:
#mege the results and get the label_str
results = np.concatenate(results)
test_idxs = test_idxs[0:len(results)]
test_label_str = [label_str[idx] for idx in test_idxs]
test_seq_len = seq_len[test_idxs]

In [9]:
len_threshold = 60
idx = 1
accuracy_score(label[test_idxs][np.logical_and(test_seq_len<=len_threshold,label[test_idxs]==idx)],
               np.argmax(results,1)[np.logical_and(test_seq_len<=len_threshold,label[test_idxs]==idx)])

0.78917117130051706

In [10]:
#append the str_idx
label_str_dict = {}
for idx, label_str in enumerate(test_label_str):
    if label_str not in label_str_dict:
        label_str_dict[label_str] = []
    label_str_dict[label_str].append(idx)
#record the result for each 
score_dict = {}
size_dict = {}
for key, value in label_str_dict.items():
    score_dict[key] = accuracy_score(label[test_idxs][np.array(value)],np.argmax(results,1)[np.array(value)])
    size_dict[key] = len(value)

In [11]:
for key in score_dict.keys():
    print('{} :  {}   {}   {}'.format(key, phrase_dict[key], size_dict[key], score_dict[key]))

[good] :  正性   47935   0.853426515072494
[哈哈] :  正性   98706   0.5982007172816243
[爱你] :  正性   34039   0.8286671171303505
[嘻嘻] :  正性   45441   0.6360555445522765
[衰] :  负性   18348   0.8109875735775016
[心] :  正性   92937   0.8795635753252203
[doge] :  负性   121986   0.7945092059744561
[威武] :  正性   9603   0.8233885244194522
[泪] :  负性   47674   0.7895708352561145
[汗] :  负性   15709   0.7906932331784328
[偷乐] :  负性   11360   0.6108274647887324
[挤眼] :  正性   5550   0.667027027027027
[鼓掌] :  正性   19318   0.8493115229319805
[拜拜] :  负性   33544   0.8834963033627474
[酷] :  正性   13983   0.6208252878495316
[阴险] :  负性   14039   0.6705605812379799
[色] :  正性   11974   0.7745114414564891
[笑哈哈] :  正性   5896   0.6648575305291723
[喵喵] :  负性   19342   0.7433564264295316
[吃惊] :  负性   29040   0.7079201101928375
[赞] :  正性   46949   0.8831498008477284
[耶] :  正性   7422   0.7890056588520614
[挖鼻] :  负性   17079   0.8231746589378769
[握手] :  正性   6536   0.9078947368421053
[玫瑰] :  正性   2724   0.9218061674008811
[鲜花] :  正性

In [15]:
document_subset = documents[test_idxs]
sample = 1
label_str_list = []
idx_list = []
for key, value in label_str_dict.items():
    label_str_list.extend([key]*sample)
    idx_list.extend(np.random.choice(label_str_dict[key],sample).tolist())
score_list = [results[:,1][idx] for idx in idx_list]
label_list = [label[test_idxs][idx] for idx in idx_list]
sent_list = [''.join([term_dict[i] for i in document_subset[idx] if i!=0]) for idx in idx_list]

In [19]:
pd.DataFrame({'emotion':label_str_list,
              'emotion_label': label_list,
              'score': score_list,
              'input_sentence': sent_list})

,emotion,emotion_label,input_sentence,score
0,[good],1,//:转发微博,0.790127
1,[哈哈],1,前呼后拥的土豪也不少啊。。。//:我只敬重实干家！大忽悠则鄙视！//：哈哈哈，但总也追星。,0.716519
2,[爱你],1,,0.699765
3,[嘻嘻],1,,0.726079
4,[衰],0,郑州：这个辖区报警后几乎不管，后来，找了保险公司，通过gps定位，通过购车辖区民警，找到车。...,0.739960
5,[心],1,在剑川过年,0.217364
6,[doge],0,看懂了呀，譬如这句“诚邀社会各界贤达暨海内外校友沥临盛举，共襄伟业，再创辉煌，特此公告”这两...,0.578732
7,[威武],1,风华正茂之际，参军报国之时。中国军视网独家首曝中国武警部队2015年宣传片。,0.960041
8,[泪],0,速度//://:下载来看！//://://://://:马下载。//:马//://:转给刚起...,0.386916
9,[汗],0,屁股不错//:屁股这么复古啊,0.485089
